In [1]:
#Importation des bibliothèque qui vont aider dans le processus de traitement
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 

#Pour manipuler les pdf
from fpdf import FPDF

#Pour récuperer la data de l'instant ou l'analyse a été produit
from datetime import datetime


# Création de la classe FPDF pour initialiser tout mes pdf(Un Constructeur en fait)

class FPDF(FPDF):
    def header(self):
        self.set_font("Arial", "B", 20)
        self.set_fill_color(255,255,255)
        self.cell(0, 10, 'Exploratory Data Analysis by Prymo',1,1,'C')
        self.set_xy(10, 30)
        self.set_font("Arial", "I", 10)
        time = datetime.now().strftime("%d/%m/%Y %H:%M")
        self.cell(w=0,h=0,txt=f"{time}")
        
    def footer(self):
        # Go to 1.5 cm from bottom
        self.set_y(-15)
        # Select Arial italic 8
        self.set_font('Arial', 'I', 15)
        # Print current and total page numbers
        #self.cell(0, 10, 'Page %s' % self.page_no() + '/{nb}', 0, 0, 'C')
        self.cell(0, 15, 'Page %s' % self.page_no() , 0, 0, 'C')





def rapport(data):
    #Distribution of network
    id_none = data[data['Network'] == 'None' ].index
    data = data.drop(id_none)
    
    #On crée un pdf en l'initialisant en dessous    
    fpdf = FPDF()
    fpdf.add_page()
    
    # Pie Plot Distribution of Network
    data['Network'].value_counts().plot(kind='pie',autopct='%.1f%%',figsize=(10,6))
    sns.set_palette('Set2')
    plt.title('Distribution of Network',
          fontname='Arial',
          fontsize=18,
          fontweight='bold'
         )
    plt.legend()
    plt.legend(bbox_to_anchor=(1, 1.02))
    plt.savefig('Images\Distribution of Network.png',dpi=200)
    plt.close()

    #On fait un saut de 10 lignes et on insère l'image 
    fpdf.ln(10)
    fpdf.image('Images\Distribution of Network.png', x=-30, y=None, w=250, h=0)
    
    #Here we are drawing the table of descriptive analysis for Network
    temp=data['Network'].value_counts().reset_index(name="N_Transactions")
    temp.loc[len(temp)] = ['Total', temp['N_Transactions'].sum()]

    fpdf.set_font('helvetica',style='I',size=18)
    fpdf.multi_cell(w=0, h=10, txt="Distribution_Of_Transaction_Network")
    fpdf.ln(5)
    fpdf.set_fill_color(255, 255, 255)
    fpdf.set_font('Arial',size=15)
    fpdf.cell(w=60, h=10, txt='Network', border=1, ln=0, fill=True,align='C')
    fpdf.cell(w=60, h=10, txt='N_Transactions', border=1, ln=1,fill=True,align='C')
    
    for i in range(0, temp.shape[0]):
        fpdf.set_fill_color(255, 255, 255)
        fpdf.cell(w=60, h=10,txt=temp['index'].iloc[i],border=1, ln=0,align='L')
        fpdf.cell(w=60, h=10,txt=temp['N_Transactions'].iloc[i].astype(str),border=1, ln=1,align='L')
    
    temp_per=round(data['Network'].value_counts(normalize=True)*100 ,1)
    fpdf.ln(10)
    fpdf.ln(30)
    fpdf.multi_cell(w=0, h=10, txt=f"The pie graph shows the actual Network of the Prymo platform , "
                                  f"must used is Airtel which has been used for {temp_per[0]}% "
                                  f"of transaction followed by Orange {temp_per[1]}%, "
                                  f" then Vodacom {temp_per[2]}% "
                                  f"and finally Africell {temp_per[3]}%.")
    
    
    
    
    
    
    #Distribution of Status
    #On crée un pdf en l'initialisant en dessous    
    fpdf.add_page()
    
    data['Status'].value_counts().plot(kind='pie',autopct='%.1f%%',figsize=(10,6))
    plt.title('Distribution of Status',
              fontname='Arial',
              fontsize=18,
              fontweight='bold'
             )
    plt.legend(bbox_to_anchor=(1, 1.02))
    plt.legend()
    plt.savefig('Images\Distribution of Status.png',dpi=200)
    plt.close()
    
    #On fait un saut de 10 lignes et on insère l'image 
    fpdf.ln(10)
    fpdf.image('Images\Distribution of Status.png', x=-30, y=None, w=250, h=0)
    
    
    #Here we are drawing the table of descriptive analysis for Network
    temp=data['Status'].value_counts().reset_index(name="Number")
    temp.loc[len(temp)] = ['Total', temp['Number'].sum()]

    fpdf.set_font('helvetica',style='I',size=18)
    fpdf.multi_cell(w=0, h=10, txt="Distribution_Of_Status")
    fpdf.ln(5)
    fpdf.set_fill_color(255, 255, 255)
    fpdf.set_font('Arial',size=15)
    fpdf.cell(w=60, h=10, txt='Status', border=1, ln=0, align='C' ,fill=True,)
    fpdf.cell(w=60, h=10, txt='Number', border=1, ln=1, align='C',fill=True,)
    
    for i in range(0, temp.shape[0]):
        fpdf.set_fill_color(255, 255, 255)
        fpdf.cell(w=60, h=10,txt=temp['index'].iloc[i],border=1, ln=0,align='L')
        fpdf.cell(w=60, h=10,txt=temp['Number'].iloc[i].astype(str),border=1, ln=1,align='L')
    
    temp_per=round(data['Status'].value_counts(normalize=True)*100 ,1)
    fpdf.ln(5)
    fpdf.multi_cell(w=0, h=10, txt=f"Here is the different numerical proportions of transactions status."
                                  f" We have {temp_per[0]}% transactions {temp['index'].iloc[0]},"
                                  f" and {temp_per[1]}% {temp['index'].iloc[1]} transactions")

    
    
    
    
    
    #AmountOfCompletedTransactionInUsd
    #On crée un pdf en l'initialisant en dessous    
    fpdf.add_page()
    completed = data.loc[(data['Status'] == 'COMPLETED') & (data['Currency'] == 'USD')]
    completed['Originator'].value_counts().sort_values(ascending=False).plot(kind='bar',
                                                                             figsize=(10,6),
                                                                             color=('green','red',
                                                                             'yellow',
                                                                             'blue',
                                                                             'orange','black')
                                                                            )

    c=completed['Originator'].value_counts()
    for i ,v in enumerate(c):
        plt.text(i,v+70, str(f'${c[i]}'),ha='center',weight='bold',size=12)


    plt.ylabel('Fees',fontname='Arial',fontsize=15)        
    plt.title('Amount of completed transactions in USD per Originator',
              fontname='Arial',
              fontsize=18,
              fontweight='bold'
             )
    plt.xticks(rotation=45)
    plt.savefig('Images\AmountCompletedUsdTransactions.png',dpi=150)
    plt.close()
    
    #On fait un saut de 10 lignes et on insère l'image 
    fpdf.ln(8)
    fpdf.image('Images\AmountCompletedUsdTransactions.png', x=-10, y=None, w=220, h=0)
    
    
    temp=completed['Originator'].value_counts().reset_index(name='Total_Amount_in_USD')
    temp.loc[len(temp)] = ['Total', temp['Total_Amount_in_USD'].sum()]
    
    fpdf.ln(5)
    fpdf.set_font('helvetica',style='I',size=15)
    fpdf.multi_cell(w=0, h=10, txt="Transactions_in_USD_completed_by_originators")
    
    fpdf.ln(5)
    fpdf.set_fill_color(255, 255, 255)
    fpdf.set_font('Arial',size=15)
    fpdf.cell(w=70, h=8, txt='Originator', border=1, ln=0, align='C' ,fill=True,)
    fpdf.cell(w=60, h=8, txt='Total_Amount_in_USD', border=1, ln=1, align='C',fill=True,)
    
    for i in range(0, temp.shape[0]):
        fpdf.set_fill_color(255, 255, 255)
        fpdf.cell(w=70, h=8,txt=temp['index'].iloc[i],border=1, ln=0,align='L')
        fpdf.cell(w=60, h=8,txt=temp['Total_Amount_in_USD'].iloc[i].astype(str),border=1, ln=1,align='L')
    
    
    
    
    
    #AmountOfCompletedTransactionPerOriginator

    #On crée un pdf en l'initialisant en dessous    
    fpdf.add_page()
    
    completed = data.loc[(data['Status'] == 'COMPLETED')]
    completed['Originator'].value_counts().sort_values(ascending=False).plot(kind='bar',
                                                                             figsize=(10,6),
                                                                             color=('green','red',
                                                                             'yellow',
                                                                             'blue',
                                                                             'orange','black')
                                                                            )

    c=completed['Originator'].value_counts()
    for i ,v in enumerate(c):
        plt.text(i,v+70, str(f'${c[i]}'),ha='center',weight='bold',size=12)


    plt.ylabel('Fees',fontname='Arial',fontsize=15)        
    plt.title('Amount of completed transactions per Originator',
              fontname='Arial',
              fontsize=18,
              fontweight='bold'
             )
    plt.xticks(rotation=45)
    plt.savefig('Images\AmountCompletedTransactions.png',dpi=200)
    plt.close()
    
    #On fait un saut de 10 lignes et on insère l'image 
    fpdf.ln(8)
    fpdf.image('Images\AmountCompletedTransactions.png', x=-10, y=None, w=230, h=0)
    
    
    temp=completed['Originator'].value_counts().reset_index(name='Total_Amount')
    temp.loc[len(temp)] = ['Total', temp['Total_Amount'].sum()]
    
    fpdf.ln(5)
    fpdf.set_font('helvetica',style='I',size=15)
    fpdf.multi_cell(w=0, h=10, txt="Transactions_completed_by_Originators")
    
    fpdf.ln(5)
    fpdf.set_fill_color(255, 255, 255)
    fpdf.set_font('Arial',size=15)
    fpdf.cell(w=70, h=8, txt='Originator', border=1, ln=0, align='C' ,fill=True,)
    fpdf.cell(w=60, h=8, txt='Total_Amount', border=1, ln=1, align='C',fill=True,)
    
    for i in range(0, temp.shape[0]):
        fpdf.set_fill_color(255, 255, 255)
        fpdf.cell(w=70, h=8,txt=temp['index'].iloc[i],border=1, ln=0,align='L')
        fpdf.cell(w=60, h=8,txt=temp['Total_Amount'].iloc[i].astype(str),border=1, ln=1,align='L')

    
    
    
    
    #FeesUsdPerOriginator
    
    #On crée un pdf en l'initialisant en dessous    
    fpdf.add_page()
    
    usd = data.loc[data['Currency'] == 'USD']
    usd['Originator'].value_counts().sort_values(ascending=False).plot(kind='bar',
                                                                             figsize=(10,6),
                                                                             color=('green','red',
                                                                             'yellow',
                                                                             'blue',
                                                                             'orange','black'),
                                                                             edgecolor='black',


                                                                        )

    u=usd['Originator'].value_counts()
    for i ,v in enumerate(u):
        plt.text(i,v+70, str(f'${u[i]}'),ha='center',weight='bold',size=12)

    plt.ylabel('Fees_USD',fontname='Arial',fontsize=15)        
    plt.title('Fees_USD per Originator',
              fontname='Arial',
              fontsize=18,
              fontweight='bold'
             )
    plt.xticks(rotation=45)
    plt.savefig('Images\Fees_USD_Originator.png',dpi=200)
    plt.close()
    
    #On fait un saut de 10 lignes et on insère l'image 
    fpdf.ln(8)
    fpdf.image('Images\Fees_USD_Originator.png', x=-10, y=None, w=230, h=0)
    
    temp=usd['Originator'].value_counts().reset_index(name='Fees_in_USD')
    temp.loc[len(temp)] = ['Total', temp['Fees_in_USD'].sum()]
    
    fpdf.ln(5)
    fpdf.set_font('helvetica',style='I',size=15)
    fpdf.multi_cell(w=0, h=10, txt="Transactions_completed_by_Originators")
    
    fpdf.ln(5)
    fpdf.set_fill_color(255, 255, 255)
    fpdf.set_font('Arial',size=15)
    fpdf.cell(w=75, h=8, txt='Originator', border=1, ln=0, align='C' ,fill=True,)
    fpdf.cell(w=60, h=8, txt='Fees_in_USD', border=1, ln=1, align='C',fill=True,)
    
    for i in range(0, temp.shape[0]):
        fpdf.set_fill_color(255, 255, 255)
        fpdf.cell(w=75, h=8,txt=temp['index'].iloc[i],border=1, ln=0,align='L')
        fpdf.cell(w=60, h=8,txt=temp['Fees_in_USD'].iloc[i].astype(str),border=1, ln=1,align='L')

    
    
    
    
    #FeesCdfPerOriginator
    
    #On crée un pdf en l'initialisant en dessous    
    fpdf.add_page()
    
    cdf = data.loc[data['Currency'] == 'CDF']
    cdf['Originator'].value_counts().sort_values(ascending=False).plot(kind='bar',
                                                                             figsize=(10,6),
                                                                             color=('green','red',
                                                                             'yellow',
                                                                             'blue',
                                                                             'orange','black'),
                                                                             edgecolor='black',
                                                                        )

    u=cdf['Originator'].value_counts()
    for i ,v in enumerate(u):
        plt.text(i,v+2, str(f'${u[i]}'),ha='center',weight='bold',size=12)

    plt.ylabel('Fees_CDF',fontname='Arial',fontsize=15)        
    plt.title('Fees_CDF per Originator',
              fontname='Arial',
              fontsize=18,
              fontweight='bold'
             )
    plt.xticks(rotation=45)
    plt.savefig('Images\Fees_CDF_Originator.png',dpi=200)
    plt.close()

    #On fait un saut de 10 lignes et on insère l'image 
    fpdf.ln(8)
    fpdf.image('Images\Fees_CDF_Originator.png', x=-10, y=None, w=230, h=0)
    
    temp=cdf['Originator'].value_counts().reset_index(name='Fees_in_CDF')
    temp.loc[len(temp)] = ['Total', temp['Fees_in_CDF'].sum()]
    
    fpdf.ln(5)
    fpdf.set_font('helvetica',style='I',size=15)
    fpdf.multi_cell(w=0, h=10, txt="Transactions_completed_by_Originators")
    
    fpdf.ln(5)
    fpdf.set_fill_color(255, 255, 255)
    fpdf.set_font('Arial',size=15)
    fpdf.cell(w=75, h=8, txt='Originator', border=1, ln=0, align='C' ,fill=True,)
    fpdf.cell(w=60, h=8, txt='Fees_in_CDF', border=1, ln=1, align='C',fill=True,)
    
    for i in range(0, temp.shape[0]):
        fpdf.set_fill_color(255, 255, 255)
        fpdf.cell(w=75, h=8,txt=temp['index'].iloc[i],border=1, ln=0,align='L')
        fpdf.cell(w=60, h=8,txt=temp['Fees_in_CDF'].iloc[i].astype(str),border=1, ln=1,align='L')

        
    
    
    
    #FeesUsdPerNetwork
    fpdf.add_page()
    
    usd = data.loc[data['Currency'] == 'USD']
    usd['Network'].value_counts().sort_values(ascending=False).plot(kind='pie',figsize=(10,6),
                                                               autopct='%.1f%%'
                                                               )
    plt.title('Fees_USD per Network',
              fontname='Arial',
              fontsize=18,
              fontweight='bold'
             )
    sns.set_palette('Set2')
    plt.legend()       
    plt.legend(bbox_to_anchor=(1, 1.02))
    plt.savefig('Images\Fees_USD_Network.png',dpi=200)
    plt.close()
    
    #On fait un saut de 10 lignes et on insère l'image 
    fpdf.ln(8)
    fpdf.image('Images\Fees_USD_Network.png', x=-10, y=None, w=230, h=0)
    
    
    temp=usd['Network'].value_counts().reset_index(name='Fees_in_USD_per_Network')
    temp.loc[len(temp)] = ['Total', temp['Fees_in_USD_per_Network'].sum()]
    
    fpdf.ln(5)
    fpdf.set_font('helvetica',style='I',size=15)
    fpdf.multi_cell(w=0, h=10, txt="Fees_in_USD_per_Network")
    
    fpdf.ln(5)
    fpdf.set_fill_color(255, 255, 255)
    fpdf.set_font('Arial',size=15)
    fpdf.cell(w=60, h=10, txt='Network', border=1, ln=0, align='C' ,fill=True,)
    fpdf.cell(w=75, h=10, txt='Fees_in_USD_per_Network', border=1, ln=1, align='C',fill=True,)
    
    for i in range(0, temp.shape[0]):
        fpdf.set_fill_color(255, 255, 255)
        fpdf.cell(w=60, h=10,txt=temp['index'].iloc[i],border=1, ln=0,align='L')
        fpdf.cell(w=75, h=10,txt=temp['Fees_in_USD_per_Network'].iloc[i].astype(str),border=1, ln=1,align='L')

    
    
    
    #FaildesTransPerOriginator
    fpdf.add_page()
    
    failed = data.loc[data['Status'] == 'FAILED']
    failed.groupby('Originator').size().plot(kind='pie',autopct='%.1f%%',figsize=(14,8))
    plt.legend()
    sns.set_palette('Set2')
    plt.legend(bbox_to_anchor=(1, 1.02))
    plt.savefig('Images\Faildes_per_Originator',dpi=200)
    plt.close()
    
    #On fait un saut de 10 lignes et on insère l'image 
    fpdf.ln(8)
    fpdf.image('Images\Faildes_per_Originator.png', x=-10, y=None, w=230, h=0)
    
    
    temp=failed['Originator'].value_counts().reset_index(name='Number_Failed_Tansactions_Originator')
    temp.loc[len(temp)] = ['Total', temp['Number_Failed_Tansactions_Originator'].sum()]
    
    fpdf.ln(5)
    fpdf.set_font('helvetica',style='I',size=15)
    fpdf.multi_cell(w=0, h=10, txt="Number_Failed_Tansactions_Originator")
    
    fpdf.ln(5)
    fpdf.set_fill_color(255, 255, 255)
    fpdf.set_font('Arial',size=15)
    fpdf.cell(w=75, h=10, txt='Originator', border=1, ln=0, align='C' ,fill=True,)
    fpdf.cell(w=100, h=10, txt='Number_Failed_Tansactions_Originator', border=1, ln=1, align='C',fill=True,)
 
    for i in range(0, temp.shape[0]):
        fpdf.set_fill_color(255, 255, 255)
        fpdf.cell(w=75, h=10,txt=temp['index'].iloc[i],border=1, ln=0,align='L')
        fpdf.cell(w=100, h=10,txt=temp['Number_Failed_Tansactions_Originator'].iloc[i].astype(str),border=1, ln=1,align='L')
    
    fpdf.output('Rapport_groupe_6.pdf')
    
    





def automatiqueReportProduction(file):
    
    #lecture du jeu de données
    data =  pd.read_excel(f'dataset/{file}')
    rapport(data)
    
automatiqueReportProduction('2024-03-21_2024-03-21_17111166632XYh.xlsx')   